In [1]:
import json 
from datetime import datetime
import typing as ty

import requests
import pandas as pd
import numpy as np

Данные матчей для профф игроков за период 1.04.2021 - 1.11.2021

In [27]:
sql_attrs = """player_matches.match_id,
player_matches.account_id,
player_matches.player_slot,
player_matches.hero_id,
player_matches.kills,
player_matches.deaths, 
player_matches.assists,
player_matches.leaver_status, 
player_matches.gold, 
player_matches.last_hits,
player_matches.denies,
player_matches.gold_per_min,
player_matches.xp_per_min,
player_matches.gold_spent,
player_matches.hero_damage,
player_matches.tower_damage,
player_matches.hero_healing,
player_matches.level,
player_matches.creeps_stacked,
player_matches.obs_placed,
player_matches.sen_placed,
player_matches.camps_stacked,
player_matches.rune_pickups,
player_matches.lane,
player_matches.lane_role,
player_matches.is_roaming,
player_matches.firstblood_claimed,
player_matches.teamfight_participation,
player_matches.towers_killed,
player_matches.roshans_killed,
player_matches.party_size,
player_matches.net_worth"""

In [28]:
sql_attrs = ', '.join([attr.strip() for attr in sql_attrs.split(',')])

In [29]:
sql_attrs

'player_matches.match_id, player_matches.account_id, player_matches.player_slot, player_matches.hero_id, player_matches.kills, player_matches.deaths, player_matches.assists, player_matches.leaver_status, player_matches.gold, player_matches.last_hits, player_matches.denies, player_matches.gold_per_min, player_matches.xp_per_min, player_matches.gold_spent, player_matches.hero_damage, player_matches.tower_damage, player_matches.hero_healing, player_matches.level, player_matches.creeps_stacked, player_matches.obs_placed, player_matches.sen_placed, player_matches.camps_stacked, player_matches.rune_pickups, player_matches.lane, player_matches.lane_role, player_matches.is_roaming, player_matches.firstblood_claimed, player_matches.teamfight_participation, player_matches.towers_killed, player_matches.roshans_killed, player_matches.party_size, player_matches.net_worth'

<h1>Data for pro players

<h3>TODO: request asynchronously

In [2]:
def get_players_stats_per_match(player_ids: ty.List[int], matches=100):
    shared_matches_list = []
    responses_status = []
    with open('player_stats.json', 'w', encoding='utf-8') as writer:
        for player_id in player_ids:
            player_recent_matches = requests.get(
                f"https://api.opendota.com/api/explorer?sql=SELECT {sql_attrs} FROM player_matches JOIN matches ON matches.match_id = player_matches.match_id WHERE player_matches.account_id = {player_id} AND matches.start_time >= extract(epoch from timestamp '2021-04-01T21:00:00.000Z') AND matches.start_time <= extract(epoch from timestamp '2021-11-01T21:00:00.000Z') AND player_matches.match_id IS NOT NULL ORDER BY matches.start_time LIMIT {matches}"
            )
            player_matches_dict = {player_id: player_recent_matches.json()}
            shared_matches_list.append(player_matches_dict)
            responses_status.append(player_recent_matches.status_code)
    
        if all([status_code == 200 for status_code in responses_status]):
            dict_to_save = json.dumps(shared_matches_list)
            writer.write(dict_to_save)
        else:
            print('Something went wrong, status code is: ', response.status_code)

<h3>Тестовая выборка игроков

In [43]:
players_ids_list = [302214028, 113331514, 321580662]

In [44]:
get_players_stats_per_match(players_ids_list)

In [48]:
with open('player_stats.json', 'r', encoding='utf-8') as file:
    players_matches = json.loads(file.read())

In [55]:
[match['match_id'] for match in players_matches[0]['302214028']['rows']][:10]

[5940129114,
 5940316763,
 5941689002,
 5941839115,
 5942003094,
 5942402971,
 5942484798,
 5948347704,
 5948486847,
 5952853218]

In [60]:
[match['match_id'] for match in players_matches[2]['321580662']['rows']][:10]

[5940129114,
 5940316763,
 5941689002,
 5941839115,
 5942003094,
 5942402971,
 5942484798,
 5948347704,
 5948486847,
 5952853218]

<h1>Data for simple hard workers

In [3]:
template = 'https://api.opendota.com/api'

In [4]:
resp = requests.get(f'{template}/playersByRank')
resp

<Response [200]>

In [5]:
resp.json()[0]

{'account_id': 268590680, 'rating': 80, 'fh_unavailable': False}

In [6]:
players = requests.get(f'{template}/players/{268590680}')
players

<Response [200]>

In [7]:
players.json()

{'tracked_until': None,
 'profile': {'account_id': 268590680,
  'personaname': '....',
  'name': None,
  'plus': False,
  'cheese': 0,
  'steamid': '76561198228856408',
  'avatar': 'https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/fc/fcba662217dbe27de744fa30ddc4fe62ed83b28a.jpg',
  'avatarmedium': 'https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/fc/fcba662217dbe27de744fa30ddc4fe62ed83b28a_medium.jpg',
  'avatarfull': 'https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/fc/fcba662217dbe27de744fa30ddc4fe62ed83b28a_full.jpg',
  'profileurl': 'https://steamcommunity.com/profiles/76561198228856408/',
  'last_login': '2019-10-12T04:46:46.150Z',
  'loccountrycode': None,
  'is_contributor': False},
 'solo_competitive_rank': 4185,
 'rank_tier': 80,
 'mmr_estimate': {'estimate': 4347},
 'leaderboard_rank': 1295,
 'competitive_rank': 3829}

<h2>Берем игроков, которые участвовали в самых последних матчах, в которых был известен средний рейтинг среди игроков

<h3>!!!Не запускать!!!

In [ ]:
public_matches = requests.get(f'{template}/publicMatches')
if public_matches.status_code == 200:
    with open(f"public_matches_{datetime.now().strftime('%Y-%m-%d')}.json", 'w', encoding='utf-8') as writer:
        writer.write(public_matches.text)
else:
    print(f"Status code: {public_matches.status_code}")

In [8]:
with open('public_matches_2021-11-19.json', 'r') as json_read:
    public_matches = json.loads(json_read.read())
public_matches[1]

{'match_id': 6281528004,
 'match_seq_num': 5250966828,
 'radiant_win': True,
 'start_time': 1637300703,
 'duration': 1355,
 'avg_mmr': 2720,
 'num_mmr': 1,
 'lobby_type': 7,
 'game_mode': 3,
 'avg_rank_tier': 42,
 'num_rank_tier': 8,
 'cluster': 227,
 'radiant_team': '34,16,54,11,20',
 'dire_team': '120,12,61,74,68'}

In [9]:
public_matches = [match for match in public_matches if match['avg_mmr'] is not None]
print(f"Средний ммр: {np.mean([match['avg_mmr'] for match in public_matches])}")

Средний ммр: 2977.9857142857145


In [10]:
len(public_matches)

70

<h2>Возьмем 10 матчей (100 игроков)

In [11]:
matches = public_matches[:10]

In [12]:
choosed_matches = []

In [13]:
matches[0]

{'match_id': 6281528311,
 'match_seq_num': 5250963470,
 'radiant_win': False,
 'start_time': 1637300723,
 'duration': 961,
 'avg_mmr': 3598,
 'num_mmr': 2,
 'lobby_type': 7,
 'game_mode': 22,
 'avg_rank_tier': 65,
 'num_rank_tier': 3,
 'cluster': 251,
 'radiant_team': '31,62,17,129,80',
 'dire_team': '70,86,111,101,106'}

In [14]:
match = requests.get(f"{template}/matches/{matches[0]['match_id']}")
match.status_code

200

In [15]:
players_ids = []
none_count = 0
list_of_matches = []
for match in matches:
    match = requests.get(f"{template}/matches/{match['match_id']}").json()
    list_of_matches.append(match)
    for player in match['players']:
        if player['account_id'] is not None:
            players_ids.append(player['account_id'])
        else:
            none_count += 1
print('Players with None account_id: ', none_count)

Players with None account_id:  49


In [16]:
len(players_ids)

51

<h2>Составление словаря матчей для каждого из игроков (по 10 матчей на игрока)

In [17]:
players_matches_dict = {}
for player_id in players_ids:
    player_data = {}
    data_to_update = []
    
    player_recent_matches = requests.get(f'{template}/players/{player_id}/recentMatches').json()
    print(player_id, len(player_recent_matches))
    players_matches_dict.update({player_id: player_recent_matches})
    

174064427 20
392188374 20
841766108 20
169994900 20
156698978 20
1009579281 20
143405256 20
904949233 20
433597293 20
444131292 20
136338447 20
1153396350 20
130733045 20
337769284 20
341047050 20
104630948 20
190191816 20
76709175 20
986082247 20
115329684 20
123654076 20
145000525 20
249520711 20
387633093 20
153908425 20
446737104 20
207983361 20
137092505 20
140291105 20
315157158 20
162421742 20
158847773 20
136186119 20
404004408 20
438103869 20
126077488 20
408064598 20
219573493 20
19075248 20
203019168 20
133098114 20
219740999 20
426240268 20
227603210 20
140740476 20
1197065565 20
213856257 20
157721965 20
308686709 20
121769131 20
198723859 20


<h2>Creating dict with average mmr for each match for each player

In [18]:
players_matches_dict[174064427][5]

{'match_id': 6271932148,
 'player_slot': 128,
 'radiant_win': True,
 'duration': 2395,
 'game_mode': 22,
 'lobby_type': 7,
 'hero_id': 121,
 'start_time': 1636777295,
 'version': None,
 'kills': 3,
 'deaths': 10,
 'assists': 8,
 'skill': 3,
 'xp_per_min': 468,
 'gold_per_min': 400,
 'hero_damage': 19228,
 'tower_damage': 589,
 'hero_healing': 0,
 'last_hits': 219,
 'lane': None,
 'lane_role': None,
 'is_roaming': None,
 'cluster': 251,
 'leaver_status': 0,
 'party_size': 5}

<h2>Тестовый игрок с ID = 174064427

In [19]:
matches_for_test_player = players_matches_dict[174064427]

In [20]:
player = requests.get(f'{template}/players/{174064427}')
player.status_code

200

In [21]:
player.json()['mmr_estimate']['estimate']

3476

In [37]:
player_id_test = 174064427
player_id_index = [player_id_test] * len(matches_for_test_player)
all_matches_df = pd.DataFrame()
columns_ = list(matches_for_test_player[0].keys())
columns_.append('player_mmr')
# columns_.append('average_match_mmr')

player_mmr = requests.get(f'{template}/players/{174064427}').json()['mmr_estimate']['estimate']
for match in matches_for_test_player:
    # calculation average mmr for each match
    # TODO refactor cause so many requests per minute (it's crashes)
    # average_match_mmr = get_average_match_mmr(match)
    
    values = list(match.values())
    values.append(player_mmr)
    # values.append(average_match_mmr)
    match_df = pd.DataFrame([values], columns=columns_, index=[player_id_test])
    all_matches_df = all_matches_df.append(match_df)
    print(f"Match {match['match_id']} has beed parsed")
# radiant_wint = all_matches_df['radiant_win'].astype(int)
# all_matches_df['radiant_win'] = radiant_wint
all_matches_df.fillna(0)

Match 6281547334 has beed parsed
Match 6281528311 has beed parsed
Match 6281499225 has beed parsed
Match 6280039217 has beed parsed
Match 6279998879 has beed parsed
Match 6271932148 has beed parsed
Match 6271885416 has beed parsed
Match 6271851560 has beed parsed
Match 6268802962 has beed parsed
Match 6268543285 has beed parsed
Match 6268509980 has beed parsed
Match 6268481440 has beed parsed
Match 6268445319 has beed parsed
Match 6268354441 has beed parsed
Match 6268273294 has beed parsed
Match 6265639899 has beed parsed
Match 6265613518 has beed parsed
Match 6265575995 has beed parsed
Match 6257149316 has beed parsed
Match 6257119179 has beed parsed


,match_id,player_slot,radiant_win,duration,game_mode,lobby_type,hero_id,start_time,version,kills,...,tower_damage,hero_healing,last_hits,lane,lane_role,is_roaming,cluster,leaver_status,party_size,player_mmr
174064427,6281547334,4,False,2493,22,7,52,1637302296,0,13,...,3310,0,332,0,0,0,251,0,2,3476
174064427,6281528311,1,False,961,22,7,129,1637300723,0,1,...,965,0,80,0,0,0,251,1,3,3476
174064427,6281499225,3,False,2013,22,7,10,1637298309,0,4,...,2896,0,309,0,0,0,251,0,3,3476
174064427,6280039217,2,False,3453,22,7,112,1637214839,0,5,...,306,1784,115,0,0,0,251,0,2,3476
174064427,6279998879,1,True,2323,22,7,25,1637211477,0,10,...,7661,110,269,0,0,0,251,0,1,3476
174064427,6271932148,128,True,2395,22,7,121,1636777295,0,3,...,589,0,219,0,0,0,251,0,5,3476
174064427,6271885416,0,True,2958,22,7,25,1636774018,0,10,...,16355,0,408,0,0,0,251,0,5,3476
174064427,6271851560,128,True,1972,22,7,98,1636771591,0,1,...,252,0,127,0,0,0,251,0,5,3476
174064427,6268802962,129,True,1496,22,7,10,1636609056,0,1,...,120,0,130,0,0,0,251,0,2,3476
174064427,6268543285,4,True,1982,22,7,120,1636585614,0,4,...,894,0,148,0,0,0,251,0,5,3476


In [105]:
all_matches_df.columns

Index(['match_id', 'player_slot', 'radiant_win', 'duration', 'game_mode',
       'lobby_type', 'hero_id', 'start_time', 'version', 'kills', 'deaths',
       'assists', 'skill', 'xp_per_min', 'gold_per_min', 'hero_damage',
       'tower_damage', 'hero_healing', 'last_hits', 'lane', 'lane_role',
       'is_roaming', 'cluster', 'leaver_status', 'party_size'],
      dtype='object')

<h2>Общий датафрейм со всеми игроками

In [33]:
def get_average_match_mmr(match_): 
    mmr_for_players = []
    if isinstance(match, requests.Response):
        match_ = match_.json()
    match_ = requests.get(f"{template}/matches/{match_['match_id']}").json()
    print(f"match_id: {match_.keys()}")
    if 'error' in match_.keys():
        return
    for match_player in match_['players']:
        if match_player['account_id'] is not None:
            player_data = requests.get(f"{template}/players/{match_player['account_id']}")
            if player_data.status_code == 200:
                if 'mmr_estimate' in player_data.json():
                    mmr_for_players.append(player_data.json()['mmr_estimate']['estimate'])
                else:
                    # print('mmr_estimate not in player_data')
                    ...
            else:
                # print("Status code: ", player_data.status_code)
                ...
        else:
            mmr_for_players.append(None)
            # print("Player count is None, continue ...")
    if any(mmr_for_players):
        return np.mean([mmr for mmr in mmr_for_players if mmr is not None])
    return 0

In [ ]:
"""player_id_test = 174064427
player_id_index = [player_id_test] * len(matches_for_test_player)
all_matches_df = pd.DataFrame()
columns_ = list(matches_for_test_player[0].keys())
columns_.append('player_mmr')
# columns_.append('average_match_mmr')

player_mmr = requests.get(f'{template}/players/{174064427}').json()['mmr_estimate']['estimate']
for match in matches_for_test_player:
    # calculation average mmr for each match
    # TODO refactor cause so many requests per minute (it's crashes)
    # average_match_mmr = get_average_match_mmr(match)
    
    values = list(match.values())
    values.append(player_mmr)
    # values.append(average_match_mmr)
    match_df = pd.DataFrame([values], columns=columns_, index=[player_id_test])
    all_matches_df = all_matches_df.append(match_df)
    print(f"Match {match['match_id']} has beed parsed")
# radiant_wint = all_matches_df['radiant_win'].astype(int)
# all_matches_df['radiant_win'] = radiant_wint
all_matches_df.fillna(0)"""

In [43]:
all_matches_df = pd.DataFrame()
columns_ = list(matches_for_test_player[0].keys())
columns_.append('player_mmr')
for player_id, matches in players_matches_dict.items():
    player_mmr = requests.get(f'{template}/players/{player_id}').json()['mmr_estimate']['estimate']
    player_id_df = player_id
    all_player_matches_df = pd.DataFrame()
    for player_match in matches:
        values = list(player_match.values())
        values.append(player_mmr)
        match_df = pd.DataFrame([values], columns=columns_, index=[player_id])
        all_player_matches_df = all_player_matches_df.append(match_df)
    all_matches_df = all_matches_df.append(all_player_matches_df)
all_matches_df   

,match_id,player_slot,radiant_win,duration,game_mode,lobby_type,hero_id,start_time,version,kills,...,tower_damage,hero_healing,last_hits,lane,lane_role,is_roaming,cluster,leaver_status,party_size,player_mmr
174064427,6281547334,4,False,2493,22,7,52,1637302296,None,13,...,3310,0,332,None,None,None,251,0,2,3476
174064427,6281528311,1,False,961,22,7,129,1637300723,None,1,...,965,0,80,None,None,None,251,1,3,3476
174064427,6281499225,3,False,2013,22,7,10,1637298309,None,4,...,2896,0,309,None,None,None,251,0,3,3476
174064427,6280039217,2,False,3453,22,7,112,1637214839,None,5,...,306,1784,115,None,None,None,251,0,2,3476
174064427,6279998879,1,True,2323,22,7,25,1637211477,None,10,...,7661,110,269,None,None,None,251,0,1,3476
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198723859,6277768373,131,True,2588,22,7,16,1637076442,None,9,...,524,0,130,None,None,None,181,0,2,2227
198723859,6277561378,3,False,3068,22,7,128,1637069206,None,20,...,2941,0,181,None,None,None,183,0,3,2227
198723859,6277462984,2,True,2070,22,7,26,1637065420,None,14,...,1948,0,62,None,None,None,181,0,3,2227
198723859,6276402485,129,False,2588,22,7,41,1636995313,None,17,...,7823,0,342,None,None,None,185,0,1,2227


In [44]:
radiant_wint = all_matches_df['radiant_win'].astype(int)
all_matches_df['radiant_win'] = radiant_wint
all_matches_df.head()

,match_id,player_slot,radiant_win,duration,game_mode,lobby_type,hero_id,start_time,version,kills,...,tower_damage,hero_healing,last_hits,lane,lane_role,is_roaming,cluster,leaver_status,party_size,player_mmr
174064427,6281547334,4,0,2493,22,7,52,1637302296,None,13,...,3310,0,332,None,None,None,251,0,2,3476
174064427,6281528311,1,0,961,22,7,129,1637300723,None,1,...,965,0,80,None,None,None,251,1,3,3476
174064427,6281499225,3,0,2013,22,7,10,1637298309,None,4,...,2896,0,309,None,None,None,251,0,3,3476
174064427,6280039217,2,0,3453,22,7,112,1637214839,None,5,...,306,1784,115,None,None,None,251,0,2,3476
174064427,6279998879,1,1,2323,22,7,25,1637211477,None,10,...,7661,110,269,None,None,None,251,0,1,3476


<h3>Dataframe с version, lane and lane_role

In [46]:
all_matches_df.to_csv('data_with_lanes.csv')

<h3>Cleaned data

In [47]:
all_matches_df.keys()

Index(['match_id', 'player_slot', 'radiant_win', 'duration', 'game_mode',
       'lobby_type', 'hero_id', 'start_time', 'version', 'kills', 'deaths',
       'assists', 'skill', 'xp_per_min', 'gold_per_min', 'hero_damage',
       'tower_damage', 'hero_healing', 'last_hits', 'lane', 'lane_role',
       'is_roaming', 'cluster', 'leaver_status', 'party_size', 'player_mmr'],
      dtype='object')

In [48]:
columns_to_drop = ['version', 'lane', 'lane_role', 'is_roaming']

In [51]:
all_matches_df = all_matches_df.drop(columns_to_drop, axis = 1)

In [58]:
all_matches_df

,match_id,player_slot,radiant_win,duration,game_mode,lobby_type,hero_id,start_time,kills,deaths,...,xp_per_min,gold_per_min,hero_damage,tower_damage,hero_healing,last_hits,cluster,leaver_status,party_size,player_mmr
174064427,6281547334,4,0,2493,22,7,52,1637302296,13,10,...,695,617,49813,3310,0,332,251,0,2,3476
174064427,6281528311,1,0,961,22,7,129,1637300723,1,4,...,345,404,4465,965,0,80,251,1,3,3476
174064427,6281499225,3,0,2013,22,7,10,1637298309,4,5,...,693,552,16606,2896,0,309,251,0,3,3476
174064427,6280039217,2,0,3453,22,7,112,1637214839,5,17,...,647,400,29715,306,1784,115,251,0,2,3476
174064427,6279998879,1,1,2323,22,7,25,1637211477,10,13,...,832,644,33193,7661,110,269,251,0,1,3476
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198723859,6277768373,131,1,2588,22,7,16,1637076442,9,12,...,637,396,35561,524,0,130,181,0,2,2227
198723859,6277561378,3,0,3068,22,7,128,1637069206,20,8,...,806,531,65994,2941,0,181,183,0,3,2227
198723859,6277462984,2,1,2070,22,7,26,1637065420,14,7,...,746,428,21112,1948,0,62,181,0,3,2227
198723859,6276402485,129,0,2588,22,7,41,1636995313,17,5,...,884,673,30498,7823,0,342,185,0,1,2227


In [57]:
all_matches_df.to_csv('data_clean.csv')

In [60]:
len(set(list(all_matches_df.index)))

51

In [45]:
all_matches_df.keys()

Index(['match_id', 'player_slot', 'radiant_win', 'duration', 'game_mode',
       'lobby_type', 'hero_id', 'start_time', 'version', 'kills', 'deaths',
       'assists', 'skill', 'xp_per_min', 'gold_per_min', 'hero_damage',
       'tower_damage', 'hero_healing', 'last_hits', 'lane', 'lane_role',
       'is_roaming', 'cluster', 'leaver_status', 'party_size', 'player_mmr'],
      dtype='object')

In [42]:
players_matches_dict[174064427]

[{'match_id': 6281547334,
  'player_slot': 4,
  'radiant_win': False,
  'duration': 2493,
  'game_mode': 22,
  'lobby_type': 7,
  'hero_id': 52,
  'start_time': 1637302296,
  'version': None,
  'kills': 13,
  'deaths': 10,
  'assists': 8,
  'skill': 3,
  'xp_per_min': 695,
  'gold_per_min': 617,
  'hero_damage': 49813,
  'tower_damage': 3310,
  'hero_healing': 0,
  'last_hits': 332,
  'lane': None,
  'lane_role': None,
  'is_roaming': None,
  'cluster': 251,
  'leaver_status': 0,
  'party_size': 2},
 {'match_id': 6281528311,
  'player_slot': 1,
  'radiant_win': False,
  'duration': 961,
  'game_mode': 22,
  'lobby_type': 7,
  'hero_id': 129,
  'start_time': 1637300723,
  'version': None,
  'kills': 1,
  'deaths': 4,
  'assists': 0,
  'skill': 3,
  'xp_per_min': 345,
  'gold_per_min': 404,
  'hero_damage': 4465,
  'tower_damage': 965,
  'hero_healing': 0,
  'last_hits': 80,
  'lane': None,
  'lane_role': None,
  'is_roaming': None,
  'cluster': 251,
  'leaver_status': 1,
  'party_size':

<h2>Inserting rows to pandas dataframe

In [67]:
columns_for_df = list(players_matches_dict[174064427][0].keys())
columns_for_df[:5]

['match_id', 'player_slot', 'radiant_win', 'duration', 'game_mode']

In [70]:
test_data = {'match_id': 6281528311, 'player_slot': 1}

In [79]:
df = pd.DataFrame(players_matches_dict[174064427][0], index=['match_id'])

In [80]:
df

,match_id,player_slot,radiant_win,duration,game_mode,lobby_type,hero_id,start_time,version,kills,...,hero_damage,tower_damage,hero_healing,last_hits,lane,lane_role,is_roaming,cluster,leaver_status,party_size
match_id,6281528311,1,False,961,22,7,129,1637300723,None,1,...,4465,965,0,80,None,None,None,251,1,3


In [41]:
type(match)

requests.models.Response

In [43]:
isinstance(match, requests.Response)

True

In [48]:
resp = requests.get(f'{template}/players/{124101446}/recentMatches')
resp.status_code

200

In [50]:
resp.json()[0]

{'match_id': 6281507718,
 'player_slot': 128,
 'radiant_win': True,
 'duration': 820,
 'game_mode': 3,
 'lobby_type': 7,
 'hero_id': 81,
 'start_time': 1637299010,
 'version': None,
 'kills': 0,
 'deaths': 3,
 'assists': 2,
 'skill': 2,
 'xp_per_min': 279,
 'gold_per_min': 294,
 'hero_damage': 3047,
 'tower_damage': 0,
 'hero_healing': 0,
 'last_hits': 45,
 'lane': None,
 'lane_role': None,
 'is_roaming': None,
 'cluster': 227,
 'leaver_status': 1,
 'party_size': 1}

In [14]:
len([match['avg_mmr'] for match in public_matches.json() if match['avg_mmr'] is not None])

75

In [38]:
match = requests.get(f'{template}/matches/6281490919')
match.status_code

200

In [113]:
match.json()['skill']

3

In [32]:
player_matches = requests.get(f'{template}/players/{124101446}/matches?limit=10')
player_matches.status_code

200

In [36]:
player_matches.json()[0]

{'match_id': 6281507718,
 'player_slot': 128,
 'radiant_win': True,
 'duration': 820,
 'game_mode': 3,
 'lobby_type': 7,
 'hero_id': 81,
 'start_time': 1637299010,
 'version': None,
 'kills': 0,
 'deaths': 3,
 'assists': 2,
 'skill': 2,
 'leaver_status': 1,
 'party_size': 1}

In [19]:
match = requests.get(f'{template}/matches/6281507718')

In [27]:
[match['account_id'] for match in match.json()['players']]

[None,
 None,
 None,
 None,
 None,
 124101446,
 915891389,
 169100142,
 185574063,
 256317901]

In [33]:
[datetime.fromtimestamp(match['start_time']).date() for match in player_matches.json()]

[datetime.date(2021, 11, 19),
 datetime.date(2021, 11, 19),
 datetime.date(2021, 11, 19),
 datetime.date(2021, 11, 18),
 datetime.date(2021, 11, 18),
 datetime.date(2021, 11, 13),
 datetime.date(2021, 11, 13),
 datetime.date(2021, 11, 13),
 datetime.date(2021, 11, 13),
 datetime.date(2021, 11, 13)]

<h2>There is not prof matches for ordinary players, so, rows object will be empty 

In [22]:
response_explorer = requests.get(f"https://api.opendota.com/api/explorer?sql=SELECT match_id FROM player_matches WHERE player_matches.account_id = 124101446 LIMIT 10")
response_explorer.status_code

200

In [24]:
response_explorer.json()['rows']

[]